In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
MISSING = -9999

In [3]:
df_train = pd.read_csv("../data/interim/train.csv")
df_test = pd.read_csv("../data/interim/test.csv")

/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
categorical = ['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161', 'cntry']

categorical2 = ['v70', 'v71', 'v72', 'v73', 'v102', 'v103','v158', 'v159', 'v160', 
                'v163', 'v164', 'v169', 'v170', 'v190', 'v191','v216', 'v231']

yes_no_cols = ['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
               'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
               'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
               'v248', 'v254', 'v256', 'v257']

education_encoding1 = ['v56', 'v58', 'v60', 'v62']
education_encoding2 = ['v57', 'v59', 'v61', 'v63']
occupation_encoding = ['v150', 'v151']

# to_drop = ['v124', 'v125', 'v126', 'v127', 'v128', 'v129', 'v130', 'v131']

In [5]:
df_train["test"] = 0
df_test["test"] = 1

In [6]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [7]:
df.shape

(39325, 274)

In [8]:
# k = df.apply(pd.to_numeric, args=('coerce',)).count().sort_values()

In [9]:
# print(k.to_string())

In [10]:
# df = df.drop(columns=list(set(k[k < 200].index).difference(categorical)))

In [11]:
# df = df.drop(columns=to_drop)

In [12]:
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["2", 2], value=0)
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["1", 1], value=1)

In [13]:
for col in education_encoding1:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({55:np.nan})

for col in education_encoding2:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({5555:np.nan})

In [14]:
for col in occupation_encoding:
    df[col] = df[col].apply(lambda x: x[:2])

In [15]:
one_hot = categorical + categorical2 + occupation_encoding

In [16]:
numeric = [col for col in df.columns if col not in one_hot + ['id', 'satisfied', 'test']]

In [17]:
# for col in numeric:
#     numeric_col = pd.to_numeric(df[col], errors='coerce')
#     na_indicator = numeric_col.isna()
#     if sum(na_indicator) > 0:
#         df[col+'_ind'] = (~na_indicator).astype(int)
#         df[col+'_missing'] = np.where(na_indicator, df[col], np.nan)
#     df[col] = numeric_col

In [18]:
# missing = [col for col in df.columns if col.endswith('_missing')]

In [19]:
for col in one_hot:
    df[col] = pd.Categorical(df[col])
df_encoded = pd.get_dummies(df, columns=one_hot, drop_first=True)

In [20]:
df_encoded = df_encoded.apply(pd.to_numeric, args=('coerce',))

In [21]:
df_encoded = df_encoded.fillna(df_encoded.min())

In [22]:
df_encoded = df_encoded.drop(columns=df_encoded.nunique()[df_encoded.nunique() <= 1].index)

In [23]:
df_encoded -= df_encoded.min()
df_encoded /= df_encoded.max()

In [24]:
df_encoded.shape

(39325, 1665)

In [25]:
df_encoded

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v151_81,v151_82,v151_83,v151_90,v151_91,v151_92,v151_93,v151_94,v151_95,v151_96
0,0.252950,0.2,0.333333,0.60,0.002325,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.651002,0.4,0.333333,0.44,0.002325,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.218467,0.6,0.333333,0.33,0.002325,0.002325,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.752518,1.0,0.333333,0.08,0.002325,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.108102,0.0,0.000000,0.10,0.002325,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39320,0.799232,0.0,0.333333,0.13,0.009483,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39321,0.098540,0.1,0.666667,0.68,0.009483,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39322,0.526625,0.5,0.000000,0.27,0.009483,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39323,0.661530,0.8,0.000000,0.05,0.009483,0.000000,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_encoded.to_csv("../data/processed/data_encoded.csv", index=False)